In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np
import csv
import sympy
from numpy import array

In [2]:
def JCU(n,PR,C_r,  p_a,B,pi,p_v=1,C_v=0,  p_E=1):
    p2 = 1-p_a
    
    NDD = pi * PR + (pi* n)
     
    JC_HRP  =  p_v*p_E*p_a*(B-pi-C_v) # RP correctly (p_E) returns answer 1 (p_a) and pay up to max_pi (-pi).
    
    NDJC_HRP_Gain = p_v*p_E*p2*(p_a**n)*(B + pi -C_v) # receives gas pi because it had to spend some to request mediation.
    NDJC_HRP_Lose = p_v*p_E*p2*(1-(p_a**n))*(B-NDD-C_r-C_v) #RP returns answer 2 correctly or responds incorrectly (p2+(1-p_E)) and send to mediator who finds out nonDeterminism (1-p_a^n), pay fine and reward (f2+r)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose
    
    DRP_Gain = p_v*(1-p_E)*(p_a**n)*(pi - C_v) 
    DRP_Lose = p_v*(1-p_E)*(1-p_a**n)*(-NDD-C_r-C_v)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p_a^n) get compensation.
#     DRP_scared = p_v*(1-p_E)*(-pi-C_v) # JC doesn't want to risk getting caught, just pay RP
#     DRP =  max(DRP_Gain+DRP_Lose, DRP_scared)
    DRP = DRP_Gain + DRP_Lose
    
    
    noID_HRP = (1-p_v)*p_E*(B-pi)
    noID_DRP = (1-p_v)*(1-p_E)*(-pi)
    noVerifyCost = noID_HRP + noID_DRP
    
    JCnonDet = JC_HRP + ND + DRP + noVerifyCost
    return JCnonDet


In [41]:
sn, sPR, sp_a, sB, spi, sp_v, sC_v, sp_E, sC_r= sympy.symbols('n PR p_a B pi p_v C_v p_E C_r', real=True)

JU = JCU(sn,sPR,sC_r,   sp_a,sB,spi,sp_v,sC_v,   sp_E)

JU = sympy.factor(JU)
display(JU)
JU = sympy.collect(JU,(spi))
display(JU)

B*p_E - C_r*p_E*p_a*p_a**n*p_v + C_r*p_E*p_a*p_v + C_r*p_a**n*p_v - C_r*p_v - C_v*p_v - PR*p_E*p_a*p_a**n*p_v*pi + PR*p_E*p_a*p_v*pi + PR*p_a**n*p_v*pi - PR*p_v*pi - n*p_E*p_a*p_a**n*p_v*pi + n*p_E*p_a*p_v*pi + n*p_a**n*p_v*pi - n*p_v*pi - p_E*p_a*p_a**n*p_v*pi - p_E*p_a*p_v*pi + p_a**n*p_v*pi + p_v*pi - pi

B*p_E - C_r*p_E*p_a*p_a**n*p_v + C_r*p_E*p_a*p_v + C_r*p_a**n*p_v - C_r*p_v - C_v*p_v + pi*(-PR*p_E*p_a*p_a**n*p_v + PR*p_E*p_a*p_v + PR*p_a**n*p_v - PR*p_v - n*p_E*p_a*p_a**n*p_v + n*p_E*p_a*p_v + n*p_a**n*p_v - n*p_v - p_E*p_a*p_a**n*p_v - p_E*p_a*p_v + p_a**n*p_v + p_v - 1)

In [4]:
LJU = JU.subs({sp_a:1 })

In [40]:
display(LJU)
LJU = sympy.collect(LJU,(spi))
display(LJU)

B*p_E - C_v*p_v - 2*p_E*p_v*pi + 2*p_v*pi - pi

B*p_E - C_v*p_v + pi*(-2*p_E*p_v + 2*p_v - 1)

In [6]:
dLJU = sympy.diff(LJU,sp_v)
display(dLJU)
dLJU = sympy.collect(dLJU,(spi*2))
display(dLJU)

-C_v - 2*p_E*pi + 2*pi

-C_v + pi*(2 - 2*p_E)

In [7]:
sol_dLJU_pE = sympy.solve([dLJU], [sp_E])
display(sol_dLJU_pE[sp_E])

(-C_v/2 + pi)/pi

In [9]:
points = 51
x,step = np.linspace(0,1,points,retstep=True)
print("step size: %s" %step)

        #-M-     -JC-                -RP-
def plot(n,PR,C_r,   pi,B,p_V,C_v,       p_E):
    
    yLJU = JCU(n=n,PR=PR,C_r=C_r,     p_a=np.ones(points),B=B,pi=pi,p_v=x,C_v=C_v,        p_E=p_E)
    
#     yLJU = list(map(lambda v: float(LJU.subs({sC_v:C_v, sB:B, sp_v:v, spi:pi, sPR:PR,  sp_E:p_E  })),x))
    
    
    plt.plot(x,yLJU, label="LJU")
    plot.plot(x, )
    
    
    plt.xlabel('p_v')
    plt.ylabel('Util') 
    plt.legend()
    plt.grid(visible=True)
    plt.show
    

step size: 0.02


In [10]:
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='PR', continuous_update=False) 
wC_r = widgets.IntSlider(min=0,max=100,step=1,value=1, description='C_r', continuous_update=False)

wpi = widgets.IntSlider(min=0,max=1000,step=1,value=1, description='pi', continuous_update=False)
wB = widgets.IntSlider(min=0,max=1000,step=1,value=2, description='B', continuous_update=False)
wp_v = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p_v', continuous_update=False) # how often JC verifies
wC_v = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='C_v',continuous_update=False) # cost for JC to verify

wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='p_E', continuous_update=False) # probability that RP runs correctly



ui1 = widgets.HBox([wn,wPR, wC_r])
ui2 = widgets.HBox([wpi,wB,wp_v,wC_v])
ui3 = widgets.HBox([wp_E])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plotFinal = widgets.interactive_output(plot,{'n':wn,'PR':wPR,'C_r':wC_r,   'pi':wpi,'B':wB,'p_V':wp_v,'C_v':wC_v,       'p_E':wp_E});

# interactive_plotFinal.layout.height = '600px'

display(interactive_plotFinal,ui1,ui2,ui3)

Output()

Lets see how much better the ND strategy is than the lazy deterministic one

In [38]:
NDgain = JU - LJU
display(NDgain)
NDgain = sympy.simplify(NDgain)
NDgain = sympy.collect(NDgain,sC_r)
NDgain = sympy.collect(NDgain,sPR)
NDgain = sympy.collect(NDgain,sn*spi)
NDgain = sympy.collect(NDgain,spi)
# NDgain = sympy.collect(NDgain,sp_E*sp_a-sp_E*sp_a**(sn+1)+sp_a**sn-1)
display(NDgain)


-C_r*p_E*p_a*p_a**n*p_v + C_r*p_E*p_a*p_v + C_r*p_a**n*p_v - C_r*p_v - PR*p_E*p_a*p_a**n*p_v*pi + PR*p_E*p_a*p_v*pi + PR*p_a**n*p_v*pi - PR*p_v*pi - n*p_E*p_a*p_a**n*p_v*pi + n*p_E*p_a*p_v*pi + n*p_a**n*p_v*pi - n*p_v*pi - p_E*p_a*p_a**n*p_v*pi - p_E*p_a*p_v*pi + 2*p_E*p_v*pi + p_a**n*p_v*pi - p_v*pi

p_v*(C_r*(p_E*p_a - p_E*p_a**(n + 1) + p_a**n - 1) + pi*(PR*(p_E*p_a - p_E*p_a**(n + 1) + p_a**n - 1) + n*(p_E*p_a - p_E*p_a**(n + 1) + p_a**n - 1) - p_E*p_a - p_E*p_a**(n + 1) + 2*p_E + p_a**n - 1))

In [52]:
NDgain_0 = NDgain.subs({sp_E:0 })
display(NDgain_0)
NDgain_1 = NDgain.subs({sp_E:1})
display(NDgain_1)

pE_check = NDgain_1 - NDgain_0
pE_check = sympy.simplify(pE_check)
print("Will lowering p_E ever increase NDG?")
display(pE_check) 
print("No. The closest it can get is when n=0")
pE_check = pE_check.subs({sn:0 })
display(pE_check) 
print("Which is 0 when pa = 1, otherwise NDgain_1 - NDgain_0 > 0 meaning that lowering p_E will not cause ND to increase.")
print("Note: This is the case because the function is linear wrt p_E \n")
print("just to double check")

NDgain_n0 = NDgain.subs({sn:0 })
display(NDgain_n0)
print("Yep. lowering p_E will not improve NDE.")


p_v*(C_r*(p_a**n - 1) + pi*(PR*(p_a**n - 1) + n*(p_a**n - 1) + p_a**n - 1))

p_v*(C_r*(p_a + p_a**n - p_a**(n + 1) - 1) + pi*(PR*(p_a + p_a**n - p_a**(n + 1) - 1) + n*(p_a + p_a**n - p_a**(n + 1) - 1) - p_a + p_a**n - p_a**(n + 1) + 1))

Will lowering p_E ever increase NDG?


p_v*(C_r*p_a - C_r*p_a**(n + 1) + PR*p_a*pi - PR*p_a**(n + 1)*pi + n*p_a*pi - n*p_a**(n + 1)*pi - p_a*pi - p_a**(n + 1)*pi + 2*pi)

No. The closest it can get is when n=0


p_v*(-2*p_a*pi + 2*pi)

Which is 0 when pa = 1, otherwise NDgain_1 - NDgain_0 > 0 meaning that lowering p_E will not cause ND to increase.
Note: This is the case because the function is linear wrt p_E 

just to double check


p_v*pi*(-2*p_E*p_a + 2*p_E)

Lets get the equation that would maximize D - H

In [93]:
dU = JU - LJU
maxdU = dU.subs({sn:1, sPR:1, sC_r:1, sp_v:1, sC_v:1, sp_E:1 })
display(maxdU)

-p_a**2 + 2*p_a + pi*(-3*p_a**2 + 4*p_a - 2) + pi - 1

In [94]:
maxdU = sympy.diff(maxdU,sp_a)
display(maxdU)

-2*p_a + pi*(4 - 6*p_a) + 2

In [95]:
sol_dmaxdU = sympy.solve([maxdU], [sp_a]) 
display(sol_dmaxdU)

{p_a: (2*pi + 1)/(3*pi + 1)}

In [96]:
maxJU = JU.subs({sn:0, sPR:0, sC_r:0, sp_v:1, sC_v:0, sp_E:1 })
display(maxJU)
# dmaxNDgain = sympy.diff(JU,sp_a)

B + pi*(1 - 2*p_a)

In [113]:
dU = JU - LJU
maxdU = dU.subs({sn:1, sPR:1, sC_r:0, sp_v:1, sC_v:0, sp_E:1 })
display(maxdU)
dmaxdU = sympy.diff(maxdU,sp_a)
display(dmaxdU)
sol_dmaxdU = sympy.solve([dmaxdU], [sp_a]) 

MAXp_a = sol_dmaxdU[sp_a]
display(type(MAXp_a))

maxdU = maxdU.subs({sp_a: MAXp_a })
display(maxdU)

pi*(-3*p_a**2 + 4*p_a - 2) + pi

pi*(4 - 6*p_a)

sympy.core.numbers.Rational

pi/3